In [ ]:
import torch
from torch.utils.data import Dataset

from modelSGNS import SkipGramModel, train_Word2Vec

In [ ]:
class manual_Dataset(Dataset):
    """ Data set qui prends les pairs directement en entré (central, pos [negs])
    """
    def __init__(self, tuples, encoder:dict, decoder:dict|None=None):
        super().__init__()
        self.encoder = encoder
        self.decoder = {i:w for w,i in self.encoder.items()}
        self.pairs = tuples

    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return torch.tensor(self.pairs[idx][0], dtype=torch.long), \
            torch.tensor(self.pairs[idx][1], dtype=torch.long), \
            torch.tensor(self.pairs[idx][2], dtype=torch.long)

    def collate_batch(self, batch):
        """Fonction de collate pour DataLoader.
        Args:
            batch: list of tuples (center, pos, neg) où neg is tensor [K]
        Return:
            Tuple(centers:Torch.Tensor [B], pos:Torch.Tensor [B], neg:Torch.Tensor [B, K])
        """
        centers = torch.stack([item[0] for item in batch], dim=0)
        pos = torch.stack([item[1] for item in batch], dim=0)
        negs = torch.stack([item[2] for item in batch], dim=0)
        return centers, pos, negs 

    def encode(self, words:list|str) -> list|int:
        if isinstance(words, str) : return self.encoder[words]
        ids = []
        for w in words :
            ids.append(self.encoder[w])
        return ids
    
    def decode(self, ids:list|int) -> list|int:
        if isinstance(ids, int) : return self.decoder[ids]
        words = []
        for i in ids :
            words.append(self.decoder[i])
        return words

In [ ]:
data1 = ['peluche', 'train', ['chat', 'chien', 'souris']]
data5 = ['tain', 'peluche', ['chat', 'chien', 'souris']]
data2 = ['chat', 'chien', ['peluche', 'train']]
data3 = ['chat', 'souris', ['peluche', 'train']]
data4 = ['chien', 'souris', ['peluche', 'train']]

encoder = {
    'peluche' : 0,
    'train' : 1,
    'chat' : 2,
    'chien' : 3,
    'souris': 4
}

nb_word = 5
list_data = [data1, data2, data3, data4, data5] * 20
print(len(list_data))